# Odor Following Navigation Algorithms Demo

This interactive notebook demonstrates gradient-following algorithms for odor plume navigation. You'll learn how agents can use simple sensory strategies to locate odor sources by testing multiple directions and moving toward higher concentrations.

## Educational Objectives

- **Understand gradient-following algorithms**: Learn how agents detect and follow odor gradients
- **Explore decision-making processes**: Visualize how agents choose directions and adjust speed
- **Parameter sensitivity analysis**: Investigate how algorithm parameters affect navigation behavior
- **Real-time algorithm visualization**: Watch agent decision-making unfold step by step

## Navigation Algorithm Overview

The odor-following algorithm implements a simple but effective strategy:

1. **Direction Testing**: Test 8 cardinal and diagonal directions (0°, 45°, 90°, 135°, 180°, 225°, 270°, 315°)
2. **Gradient Detection**: Compare odor concentrations at potential movement positions
3. **Direction Selection**: Choose the direction with the highest odor concentration
4. **Adaptive Speed Control**: Adjust movement speed based on gradient strength
5. **Stopping Criteria**: Halt when reaching sufficiently strong odor sources (>0.9 concentration)

This approach mimics real biological navigation strategies used by insects and marine animals.

## Setup and Imports

First, let's import the necessary modules using the new project structure and configure our environment for reproducible experiments.

In [ ]:
# Standard library imports
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from typing import Tuple, List, Dict, Optional
from pathlib import Path

# Interactive widgets for parameter exploration
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from ipywidgets import interact, interactive, fixed, interact_manual

# Project imports using new structure
from {{cookiecutter.project_slug}}.core.navigator import NavigatorProtocol
from {{cookiecutter.project_slug}}.utils.visualization import SimulationVisualization, visualize_trajectory
from {{cookiecutter.project_slug}}.utils.seed_manager import set_global_seed, get_current_seed, SeedConfig
from {{cookiecutter.project_slug}}.config.schemas import NavigatorConfig, SingleAgentConfig

# Hydra configuration imports
try:
    from omegaconf import DictConfig, OmegaConf
    HYDRA_AVAILABLE = True
except ImportError:
    HYDRA_AVAILABLE = False
    print("Hydra not available - using fallback configuration")

# Configure matplotlib for notebook
%matplotlib widget
plt.style.use('default')  # Clean, publication-ready plots

print("✅ All imports successful!")
print(f"📊 Matplotlib backend: {plt.get_backend()}")
print(f"🎛️ Interactive widgets available: {widgets.__version__}")

## Reproducible Experiment Setup

We'll configure reproducible random number generation to ensure consistent results across different runs and computing environments.

In [ ]:
# Configure reproducible experiment environment
DEMO_SEED = 42  # Fixed seed for consistent educational examples

# Initialize seed management with comprehensive configuration
seed_config = SeedConfig(
    seed=DEMO_SEED,
    validate_initialization=True,
    preserve_state=True,  # Enable state preservation for parameter experiments
    log_seed_context=True,
    hash_environment=True  # Include environment characteristics for consistency
)

# Set global seed and get confirmation
initialized_seed = set_global_seed(config=seed_config)
print(f"🌱 Global seed initialized: {initialized_seed}")
print(f"🎲 Current seed: {get_current_seed()}")

# Test reproducibility with sample values
initial_samples = {
    'numpy_random': np.random.random(),
    'numpy_normal': np.random.normal(0, 1),
    'numpy_integers': np.random.randint(0, 100, size=3).tolist()
}

print("\n📋 Reproducibility verification samples:")
for key, value in initial_samples.items():
    print(f"  {key}: {value}")

print("\n✅ Reproducible environment configured successfully!")

## Algorithm Configuration and Parameters

Let's define the configurable parameters for our odor-following algorithm. These parameters control the agent's behavior and can be adjusted to explore different navigation strategies.

In [ ]:
# Algorithm configuration using Hydra-compatible schema
default_config = {
    "algorithm": {
        "test_directions": [0, 45, 90, 135, 180, 225, 270, 315],  # Cardinal and diagonal directions
        "test_distance": 2.0,  # Distance to test ahead for odor comparison
        "base_speed": 0.3,     # Minimum movement speed
        "speed_scaling": 2.0,  # Multiplier for gradient-based speed adjustment
        "convergence_threshold": 0.9,  # Odor concentration threshold for stopping
        "max_steps": 40,       # Maximum simulation steps
        "time_step": 0.5       # Simulation time step size
    },
    "agent": {
        "start_position": [5, 5],    # Starting position [x, y]
        "start_orientation": 0,      # Initial orientation in degrees
        "max_speed": 1.0            # Maximum allowed speed
    },
    "environment": {
        "width": 50,
        "height": 50,
        "odor_sources": [
            # [x, y, intensity, sigma] for each Gaussian odor source
            [35, 30, 1.0, 5.0],   # Primary odor source
            [15, 15, 0.7, 7.0]    # Secondary odor source
        ]
    },
    "visualization": {
        "figsize": [12, 10],
        "animation_interval": 300,  # milliseconds between frames
        "trail_length": 20,        # Number of trail points to show
        "show_direction_tests": True,  # Visualize direction testing
        "show_decision_process": True  # Show agent decision-making annotations
    }
}

# Convert to Hydra-compatible format if available
if HYDRA_AVAILABLE:
    demo_config = OmegaConf.create(default_config)
    print("🔧 Configuration loaded with Hydra OmegaConf")
else:
    demo_config = default_config
    print("🔧 Configuration loaded with standard dictionary")

print("\n📋 Algorithm Configuration:")
print(f"  🧭 Test directions: {demo_config['algorithm']['test_directions']}")
print(f"  📏 Test distance: {demo_config['algorithm']['test_distance']} units")
print(f"  🏃 Speed range: {demo_config['algorithm']['base_speed']} - {demo_config['agent']['max_speed']} units/step")
print(f"  🎯 Convergence threshold: {demo_config['algorithm']['convergence_threshold']}")
print(f"  🌍 Environment size: {demo_config['environment']['width']}×{demo_config['environment']['height']}")
print(f"  🗺️ Odor sources: {len(demo_config['environment']['odor_sources'])} sources")

## Environment Creation: Gaussian Odor Fields

We'll create a realistic odor environment with multiple Gaussian concentration sources. This simulates how odor plumes spread and mix in natural environments.

In [ ]:
def create_gaussian_odor_field(
    width: int, 
    height: int, 
    odor_sources: List[List[float]],
    normalize: bool = True
) -> np.ndarray:
    """
    Create a realistic Gaussian odor field with multiple sources.
    
    This function creates a 2D odor concentration field by superimposing
    multiple Gaussian distributions, each representing an odor source.
    
    Args:
        width: Environment width in grid units
        height: Environment height in grid units
        odor_sources: List of [x, y, intensity, sigma] for each source
        normalize: Whether to normalize maximum concentration to 1.0
    
    Returns:
        2D numpy array with odor concentration values
    """
    # Create coordinate meshgrid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    odor_field = np.zeros((height, width), dtype=np.float32)
    
    # Add each Gaussian odor source
    for source_x, source_y, intensity, sigma in odor_sources:
        # Calculate Gaussian distribution centered at (source_x, source_y)
        gaussian = np.exp(-((x - source_x)**2 + (y - source_y)**2) / (2 * sigma**2))
        odor_field += intensity * gaussian
    
    # Normalize to [0, 1] range if requested
    if normalize:
        max_val = np.max(odor_field)
        if max_val > 0:
            odor_field /= max_val
    
    return odor_field

# Create the demo odor environment
odor_environment = create_gaussian_odor_field(
    width=demo_config['environment']['width'],
    height=demo_config['environment']['height'],
    odor_sources=demo_config['environment']['odor_sources']
)

# Visualize the odor environment
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(
    odor_environment, 
    origin='lower', 
    cmap='viridis', 
    extent=[0, demo_config['environment']['width'], 0, demo_config['environment']['height']]
)

# Mark odor sources
for i, (x, y, intensity, sigma) in enumerate(demo_config['environment']['odor_sources']):
    ax.scatter(x, y, color='red', s=100, marker='x', linewidth=3, 
              label=f'Source {i+1} (I={intensity:.1f}, σ={sigma:.1f})')

# Mark agent start position
start_x, start_y = demo_config['agent']['start_position']
ax.scatter(start_x, start_y, color='white', s=150, marker='o', 
          edgecolors='black', linewidth=2, label='Agent Start')

ax.set_xlabel('X Position', fontsize=12)
ax.set_ylabel('Y Position', fontsize=12)
ax.set_title('Odor Environment with Gaussian Sources', fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Odor Concentration', fontsize=12)

plt.tight_layout()
plt.show()

# Environment statistics
print(f"\n🌍 Environment Statistics:")
print(f"  📏 Dimensions: {odor_environment.shape}")
print(f"  📈 Concentration range: {np.min(odor_environment):.3f} - {np.max(odor_environment):.3f}")
print(f"  🎯 Peak locations: {np.unravel_index(np.argmax(odor_environment), odor_environment.shape)}")
print(f"  📊 Mean concentration: {np.mean(odor_environment):.3f}")

## Simple Navigator Implementation

We'll implement a simplified navigator that follows the NavigatorProtocol interface. This demonstrates how the gradient-following algorithm integrates with the broader navigation framework.

In [ ]:
class SimpleOdorFollowingNavigator:
    """
    Simplified navigator implementing gradient-following odor navigation.
    
    This class demonstrates the core algorithm for educational purposes,
    implementing key methods from NavigatorProtocol while focusing on
    clarity and educational value.
    """
    
    def __init__(self, config: Dict):
        """Initialize navigator with configuration parameters."""
        self.config = config
        
        # Agent state variables
        self.position = np.array(config['agent']['start_position'], dtype=float)
        self.orientation = float(config['agent']['start_orientation'])
        self.speed = 0.0  # Start with zero speed
        self.max_speed = config['agent']['max_speed']
        
        # Algorithm parameters
        self.test_directions = config['algorithm']['test_directions']
        self.test_distance = config['algorithm']['test_distance']
        self.base_speed = config['algorithm']['base_speed']
        self.speed_scaling = config['algorithm']['speed_scaling']
        self.convergence_threshold = config['algorithm']['convergence_threshold']
        
        # Tracking variables for visualization
        self.last_decision = None
        self.direction_tests = []
        self.trajectory = [self.position.copy()]
        self.orientation_history = [self.orientation]
        self.speed_history = [self.speed]
        
    @property
    def positions(self) -> np.ndarray:
        """Get current position as NavigatorProtocol expects."""
        return self.position.reshape(1, -1)
    
    @property
    def orientations(self) -> np.ndarray:
        """Get current orientation as NavigatorProtocol expects."""
        return np.array([self.orientation])
    
    @property
    def speeds(self) -> np.ndarray:
        """Get current speed as NavigatorProtocol expects."""
        return np.array([self.speed])
    
    def sample_odor(self, env_array: np.ndarray) -> float:
        """Sample odor concentration at current position using bilinear interpolation."""
        x, y = self.position
        height, width = env_array.shape
        
        # Boundary checks
        if x < 0 or x >= width or y < 0 or y >= height:
            return 0.0
        
        # Bilinear interpolation for sub-pixel accuracy
        x_int, y_int = int(x), int(y)
        x_frac, y_frac = x - x_int, y - y_int
        
        # Clamp to array bounds
        x_int = min(x_int, width - 1)
        y_int = min(y_int, height - 1)
        x_next = min(x_int + 1, width - 1)
        y_next = min(y_int + 1, height - 1)
        
        # Interpolate
        top_left = env_array[y_int, x_int]
        top_right = env_array[y_int, x_next]
        bottom_left = env_array[y_next, x_int]
        bottom_right = env_array[y_next, x_next]
        
        # Bilinear interpolation formula
        top = top_left * (1 - x_frac) + top_right * x_frac
        bottom = bottom_left * (1 - x_frac) + bottom_right * x_frac
        result = top * (1 - y_frac) + bottom * y_frac
        
        return float(result)
    
    def test_direction(self, env_array: np.ndarray, direction_deg: float) -> float:
        """Test odor concentration at a position offset in the given direction."""
        # Calculate test position
        direction_rad = np.radians(direction_deg)
        test_x = self.position[0] + self.test_distance * np.cos(direction_rad)
        test_y = self.position[1] + self.test_distance * np.sin(direction_rad)
        
        # Create temporary position for sampling
        original_pos = self.position.copy()
        self.position = np.array([test_x, test_y])
        
        # Sample odor at test position
        test_odor = self.sample_odor(env_array)
        
        # Restore original position
        self.position = original_pos
        
        return test_odor
    
    def make_navigation_decision(self, env_array: np.ndarray) -> Dict:
        """
        Core gradient-following algorithm: test all directions and choose the best.
        
        Returns detailed decision information for visualization and analysis.
        """
        current_odor = self.sample_odor(env_array)
        
        # Test all directions
        direction_tests = []
        best_direction = self.orientation  # Default: keep current direction
        best_odor = current_odor
        
        for test_direction in self.test_directions:
            test_odor = self.test_direction(env_array, test_direction)
            direction_tests.append({
                'direction': test_direction,
                'odor': test_odor,
                'improvement': test_odor - current_odor
            })
            
            # Update best direction if this is better
            if test_odor > best_odor:
                best_odor = test_odor
                best_direction = test_direction
        
        # Calculate speed based on odor gradient
        odor_improvement = max(0, best_odor - current_odor)
        new_speed = min(
            self.base_speed + odor_improvement * self.speed_scaling,
            self.max_speed
        )
        
        # Create decision summary
        decision = {
            'current_odor': current_odor,
            'best_odor': best_odor,
            'odor_improvement': odor_improvement,
            'chosen_direction': best_direction,
            'new_speed': new_speed,
            'direction_tests': direction_tests,
            'converged': current_odor >= self.convergence_threshold
        }
        
        # Update agent state
        self.orientation = best_direction
        self.speed = new_speed
        self.last_decision = decision
        self.direction_tests = direction_tests
        
        return decision
    
    def step(self, env_array: np.ndarray, dt: float = 0.5) -> Dict:
        """Execute one navigation step and return movement information."""
        # Make navigation decision
        decision = self.make_navigation_decision(env_array)
        
        # Don't move if converged
        if decision['converged']:
            step_info = {
                'moved': False,
                'reason': 'converged',
                'decision': decision
            }
        else:
            # Calculate movement vector
            direction_rad = np.radians(self.orientation)
            dx = self.speed * dt * np.cos(direction_rad)
            dy = self.speed * dt * np.sin(direction_rad)
            
            # Update position
            self.position += np.array([dx, dy])
            
            step_info = {
                'moved': True,
                'displacement': np.array([dx, dy]),
                'decision': decision
            }
        
        # Update tracking history
        self.trajectory.append(self.position.copy())
        self.orientation_history.append(self.orientation)
        self.speed_history.append(self.speed)
        
        return step_info
    
    def reset(self):
        """Reset navigator to initial state."""
        self.position = np.array(self.config['agent']['start_position'], dtype=float)
        self.orientation = float(self.config['agent']['start_orientation'])
        self.speed = 0.0
        self.last_decision = None
        self.direction_tests = []
        self.trajectory = [self.position.copy()]
        self.orientation_history = [self.orientation]
        self.speed_history = [self.speed]

print("🤖 SimpleOdorFollowingNavigator class defined successfully!")
print("📋 Key features:")
print("  • 8-direction gradient testing")
print("  • Adaptive speed control based on gradient strength")
print("  • Bilinear interpolation for accurate odor sampling")
print("  • Comprehensive decision tracking for visualization")
print("  • NavigatorProtocol-compatible interface")

## Interactive Algorithm Demonstration

Now let's run the complete odor-following algorithm and visualize how the agent makes decisions step by step.

In [ ]:
def run_odor_following_demo(config: Dict, verbose: bool = True) -> Dict:
    """
    Run complete odor-following demonstration with detailed logging.
    
    Args:
        config: Algorithm configuration dictionary
        verbose: Whether to print step-by-step information
    
    Returns:
        Dictionary containing complete simulation results
    """
    # Create navigator
    navigator = SimpleOdorFollowingNavigator(config)
    
    # Create environment
    env = create_gaussian_odor_field(
        width=config['environment']['width'],
        height=config['environment']['height'],
        odor_sources=config['environment']['odor_sources']
    )
    
    # Simulation variables
    max_steps = config['algorithm']['max_steps']
    dt = config['algorithm']['time_step']
    step_results = []
    
    if verbose:
        print(f"🚀 Starting odor-following simulation...")
        print(f"📍 Start position: {navigator.position}")
        print(f"🧭 Start orientation: {navigator.orientation}°")
        print(f"🎯 Convergence threshold: {navigator.convergence_threshold}")
        print("\n" + "="*60)
    
    # Main simulation loop
    for step_num in range(max_steps):
        step_info = navigator.step(env, dt)
        step_results.append(step_info)
        
        if verbose:
            decision = step_info['decision']
            print(f"\n📊 Step {step_num + 1:2d}:")
            print(f"  📍 Position: [{navigator.position[0]:5.1f}, {navigator.position[1]:5.1f}]")
            print(f"  🧭 Direction: {navigator.orientation:6.1f}° → {decision['chosen_direction']:6.1f}°")
            print(f"  🏃 Speed: {decision['new_speed']:.2f} units/step")
            print(f"  👃 Odor: {decision['current_odor']:.3f} → {decision['best_odor']:.3f} (+{decision['odor_improvement']:.3f})")
            
            if decision['converged']:
                print(f"  🎯 CONVERGENCE ACHIEVED! Odor = {decision['current_odor']:.3f} ≥ {navigator.convergence_threshold}")
                break
            
            if not step_info['moved']:
                print(f"  ⏸️  Agent stopped: {step_info['reason']}")
    
    # Compile final results
    results = {
        'navigator': navigator,
        'environment': env,
        'step_results': step_results,
        'final_position': navigator.position.copy(),
        'trajectory': np.array(navigator.trajectory),
        'orientations': np.array(navigator.orientation_history),
        'speeds': np.array(navigator.speed_history),
        'total_steps': len(step_results),
        'converged': step_results[-1]['decision']['converged'] if step_results else False,
        'final_odor': step_results[-1]['decision']['current_odor'] if step_results else 0.0
    }
    
    if verbose:
        print("\n" + "="*60)
        print(f"✅ Simulation completed!")
        print(f"📊 Total steps: {results['total_steps']}")
        print(f"📍 Final position: [{results['final_position'][0]:.1f}, {results['final_position'][1]:.1f}]")
        print(f"👃 Final odor concentration: {results['final_odor']:.3f}")
        print(f"🎯 Converged: {'✅ YES' if results['converged'] else '❌ NO'}")
        
        # Calculate trajectory statistics
        trajectory_length = np.sum(np.linalg.norm(np.diff(results['trajectory'], axis=0), axis=1))
        direct_distance = np.linalg.norm(results['final_position'] - results['trajectory'][0])
        efficiency = direct_distance / trajectory_length if trajectory_length > 0 else 0
        
        print(f"📏 Trajectory length: {trajectory_length:.1f} units")
        print(f"📐 Direct distance: {direct_distance:.1f} units")
        print(f"⚡ Path efficiency: {efficiency:.2f}")
    
    return results

# Run the demonstration
print("🎬 Running Odor Following Demonstration")
print("=====================================\n")

demo_results = run_odor_following_demo(demo_config, verbose=True)

## Detailed Decision Visualization

Let's create comprehensive visualizations showing the agent's decision-making process, including direction testing and gradient detection.

In [ ]:
def visualize_navigation_decisions(results: Dict, step_indices: List[int] = None):
    """
    Create detailed visualization of agent navigation decisions.
    
    Shows the agent's decision-making process including direction testing,
    gradient detection, and movement choices for selected simulation steps.
    """
    navigator = results['navigator']
    environment = results['environment']
    step_results = results['step_results']
    
    # Default to visualizing key steps if not specified
    if step_indices is None:
        total_steps = len(step_results)
        step_indices = [0, total_steps//4, total_steps//2, 3*total_steps//4, total_steps-1]
        step_indices = [i for i in step_indices if i < total_steps]
    
    # Create subplot layout
    n_steps = len(step_indices)
    fig, axes = plt.subplots(2, n_steps, figsize=(4*n_steps, 8))
    if n_steps == 1:
        axes = axes.reshape(2, 1)
    
    trajectory = results['trajectory']
    
    for col, step_idx in enumerate(step_indices):
        step_info = step_results[step_idx]
        decision = step_info['decision']
        position = trajectory[step_idx]
        
        # Top subplot: Environment view with decision visualization
        ax_env = axes[0, col]
        
        # Show environment
        im = ax_env.imshow(
            environment, 
            origin='lower', 
            cmap='viridis', 
            alpha=0.7,
            extent=[0, environment.shape[1], 0, environment.shape[0]]
        )
        
        # Show trajectory up to current step
        if step_idx > 0:
            ax_env.plot(
                trajectory[:step_idx+1, 0], 
                trajectory[:step_idx+1, 1],
                'w-', linewidth=2, alpha=0.8, label='Trajectory'
            )
        
        # Show current position
        ax_env.scatter(
            position[0], position[1], 
            color='red', s=150, marker='o', 
            edgecolors='white', linewidth=2,
            label='Current Position', zorder=10
        )
        
        # Visualize direction tests
        test_distance = navigator.test_distance
        for test in decision['direction_tests']:
            direction_rad = np.radians(test['direction'])
            test_x = position[0] + test_distance * np.cos(direction_rad)
            test_y = position[1] + test_distance * np.sin(direction_rad)
            
            # Color based on odor improvement
            if test['improvement'] > 0:
                color = 'lime'  # Positive gradient
                size = 100 + test['improvement'] * 200
            else:
                color = 'orange'  # Negative or no gradient
                size = 50
            
            # Draw arrow from current position to test position
            ax_env.annotate(
                '', xy=(test_x, test_y), xytext=position,
                arrowprops=dict(
                    arrowstyle='->', color=color, alpha=0.7, lw=2
                ),
                zorder=5
            )
            
            # Mark test position
            ax_env.scatter(
                test_x, test_y, color=color, s=size, alpha=0.7,
                edgecolors='black', linewidth=1, zorder=8
            )
        
        # Highlight chosen direction
        chosen_rad = np.radians(decision['chosen_direction'])
        chosen_x = position[0] + test_distance * np.cos(chosen_rad)
        chosen_y = position[1] + test_distance * np.sin(chosen_rad)
        
        ax_env.annotate(
            '', xy=(chosen_x, chosen_y), xytext=position,
            arrowprops=dict(
                arrowstyle='->', color='yellow', lw=4, alpha=0.9
            ),
            zorder=15
        )
        
        ax_env.set_title(
            f'Step {step_idx + 1}\nOdor: {decision["current_odor"]:.3f} → {decision["best_odor"]:.3f}',
            fontsize=10
        )
        ax_env.set_xlabel('X Position')
        if col == 0:
            ax_env.set_ylabel('Y Position')
        
        # Add legend only to first subplot
        if col == 0:
            ax_env.legend(loc='upper left', fontsize=8)
        
        # Bottom subplot: Decision analysis
        ax_decision = axes[1, col]
        
        # Extract direction test data
        directions = [test['direction'] for test in decision['direction_tests']]
        odor_values = [test['odor'] for test in decision['direction_tests']]
        improvements = [test['improvement'] for test in decision['direction_tests']]
        
        # Create polar-style bar chart
        theta = np.radians(directions)
        
        # Color bars based on improvement
        colors = ['lime' if imp > 0 else 'orange' for imp in improvements]
        
        # Highlight chosen direction
        chosen_idx = directions.index(decision['chosen_direction'])
        colors[chosen_idx] = 'yellow'
        
        # Create bar chart
        bars = ax_decision.bar(
            directions, odor_values, 
            color=colors, alpha=0.7, 
            edgecolor='black', linewidth=1
        )
        
        # Highlight chosen direction with thicker border
        bars[chosen_idx].set_edgecolor('red')
        bars[chosen_idx].set_linewidth(3)
        
        ax_decision.set_xlabel('Test Direction (degrees)')
        ax_decision.set_ylabel('Odor Concentration')
        ax_decision.set_title(f'Direction Analysis\nChosen: {decision["chosen_direction"]}°')
        ax_decision.grid(True, alpha=0.3)
        
        # Add current odor reference line
        ax_decision.axhline(
            decision['current_odor'], 
            color='red', linestyle='--', alpha=0.7,
            label='Current Odor'
        )
        
        if col == 0:
            ax_decision.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print decision summary
    print("\n🔍 Decision Analysis Summary:")
    for i, step_idx in enumerate(step_indices):
        decision = step_results[step_idx]['decision']
        print(f"\n📊 Step {step_idx + 1}:")
        print(f"  🎯 Best direction: {decision['chosen_direction']}°")
        print(f"  📈 Odor improvement: +{decision['odor_improvement']:.3f}")
        print(f"  🏃 Speed adjustment: {decision['new_speed']:.2f}")
        print(f"  ✅ Converged: {'Yes' if decision['converged'] else 'No'}")

# Visualize decision-making process
print("🔍 Analyzing Agent Decision-Making Process")
print("==========================================\n")

visualize_navigation_decisions(demo_results)

## Interactive Parameter Exploration

Now let's create interactive widgets to explore how different algorithm parameters affect navigation behavior. This allows real-time experimentation with the odor-following strategy.

In [ ]:
# Parameter exploration widgets
def create_parameter_exploration_interface():
    """
    Create interactive widgets for real-time parameter exploration.
    
    This function sets up sliders and controls that allow users to modify
    algorithm parameters and immediately see their effects on navigation behavior.
    """
    
    # Define parameter widgets
    widgets_dict = {
        'test_distance': widgets.FloatSlider(
            value=2.0, min=0.5, max=5.0, step=0.1,
            description='Test Distance:',
            tooltip='Distance ahead to test for odor concentration'
        ),
        'base_speed': widgets.FloatSlider(
            value=0.3, min=0.1, max=1.0, step=0.05,
            description='Base Speed:',
            tooltip='Minimum movement speed'
        ),
        'speed_scaling': widgets.FloatSlider(
            value=2.0, min=0.5, max=5.0, step=0.1,
            description='Speed Scaling:',
            tooltip='Multiplier for gradient-based speed adjustment'
        ),
        'convergence_threshold': widgets.FloatSlider(
            value=0.9, min=0.5, max=1.0, step=0.01,
            description='Convergence:',
            tooltip='Odor concentration threshold for stopping'
        ),
        'max_steps': widgets.IntSlider(
            value=40, min=10, max=100, step=5,
            description='Max Steps:',
            tooltip='Maximum simulation steps'
        ),
        'start_x': widgets.FloatSlider(
            value=5, min=0, max=25, step=1,
            description='Start X:',
            tooltip='Agent starting X position'
        ),
        'start_y': widgets.FloatSlider(
            value=5, min=0, max=25, step=1,
            description='Start Y:',
            tooltip='Agent starting Y position'
        ),
        'start_orientation': widgets.FloatSlider(
            value=0, min=0, max=359, step=15,
            description='Start Angle:',
            tooltip='Initial orientation in degrees'
        )
    }
    
    # Output widget for results
    output = widgets.Output()
    
    def run_parameter_experiment(**kwargs):
        """Run simulation with updated parameters and display results."""
        with output:
            clear_output(wait=True)
            
            print("🔄 Running experiment with updated parameters...")
            
            # Create modified configuration
            modified_config = demo_config.copy() if hasattr(demo_config, 'copy') else dict(demo_config)
            
            # Update algorithm parameters
            if hasattr(modified_config, '__setitem__'):  # Dictionary-like
                modified_config['algorithm']['test_distance'] = kwargs['test_distance']
                modified_config['algorithm']['base_speed'] = kwargs['base_speed']
                modified_config['algorithm']['speed_scaling'] = kwargs['speed_scaling']
                modified_config['algorithm']['convergence_threshold'] = kwargs['convergence_threshold']
                modified_config['algorithm']['max_steps'] = kwargs['max_steps']
                modified_config['agent']['start_position'] = [kwargs['start_x'], kwargs['start_y']]
                modified_config['agent']['start_orientation'] = kwargs['start_orientation']
            else:  # OmegaConf object
                modified_config.algorithm.test_distance = kwargs['test_distance']
                modified_config.algorithm.base_speed = kwargs['base_speed']
                modified_config.algorithm.speed_scaling = kwargs['speed_scaling']
                modified_config.algorithm.convergence_threshold = kwargs['convergence_threshold']
                modified_config.algorithm.max_steps = kwargs['max_steps']
                modified_config.agent.start_position = [kwargs['start_x'], kwargs['start_y']]
                modified_config.agent.start_orientation = kwargs['start_orientation']
            
            # Run experiment
            try:
                results = run_odor_following_demo(modified_config, verbose=False)
                
                # Display compact results
                print(f"✅ Experiment completed!")
                print(f"📊 Steps taken: {results['total_steps']}/{kwargs['max_steps']}")
                print(f"📍 Final position: [{results['final_position'][0]:.1f}, {results['final_position'][1]:.1f}]")
                print(f"👃 Final odor: {results['final_odor']:.3f}")
                print(f"🎯 Converged: {'✅ YES' if results['converged'] else '❌ NO'}")
                
                # Calculate efficiency metrics
                trajectory = results['trajectory']
                if len(trajectory) > 1:
                    path_length = np.sum(np.linalg.norm(np.diff(trajectory, axis=0), axis=1))
                    direct_distance = np.linalg.norm(results['final_position'] - trajectory[0])
                    efficiency = direct_distance / path_length if path_length > 0 else 0
                    print(f"⚡ Path efficiency: {efficiency:.2f}")
                    print(f"📏 Path length: {path_length:.1f} units")
                
                # Quick trajectory visualization
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
                
                # Environment with trajectory
                im = ax1.imshow(
                    results['environment'], origin='lower', cmap='viridis', alpha=0.7,
                    extent=[0, results['environment'].shape[1], 0, results['environment'].shape[0]]
                )
                ax1.plot(trajectory[:, 0], trajectory[:, 1], 'w-', linewidth=3, alpha=0.9)
                ax1.scatter(trajectory[0, 0], trajectory[0, 1], color='green', s=150, marker='o', label='Start')
                ax1.scatter(trajectory[-1, 0], trajectory[-1, 1], color='red', s=150, marker='s', label='End')
                ax1.set_title('Trajectory Visualization')
                ax1.legend()
                plt.colorbar(im, ax=ax1, label='Odor Concentration')
                
                # Performance metrics over time
                steps = range(len(results['speeds']))
                ax2_twin = ax2.twinx()
                
                line1 = ax2.plot(steps, results['speeds'], 'b-', label='Speed', linewidth=2)
                ax2.set_ylabel('Speed (units/step)', color='b')
                ax2.tick_params(axis='y', labelcolor='b')
                
                # Calculate odor concentration over time
                odor_history = []
                for pos in trajectory:
                    navigator_temp = SimpleOdorFollowingNavigator(modified_config)
                    navigator_temp.position = pos
                    odor_history.append(navigator_temp.sample_odor(results['environment']))
                
                line2 = ax2_twin.plot(steps, odor_history, 'r-', label='Odor Concentration', linewidth=2)
                ax2_twin.set_ylabel('Odor Concentration', color='r')
                ax2_twin.tick_params(axis='y', labelcolor='r')
                ax2_twin.axhline(kwargs['convergence_threshold'], color='r', linestyle='--', alpha=0.7, label='Threshold')
                
                ax2.set_xlabel('Simulation Step')
                ax2.set_title('Performance Metrics')
                ax2.grid(True, alpha=0.3)
                
                # Combined legend
                lines1, labels1 = ax2.get_legend_handles_labels()
                lines2, labels2 = ax2_twin.get_legend_handles_labels()
                ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
                
                plt.tight_layout()
                plt.show()
                
            except Exception as e:
                print(f"❌ Error running experiment: {e}")
                import traceback
                traceback.print_exc()
    
    # Create interactive interface
    interactive_plot = interactive(run_parameter_experiment, **widgets_dict)
    
    # Layout
    parameter_box = widgets.VBox([
        widgets.HTML("<h3>🎛️ Algorithm Parameters</h3>"),
        widgets.HBox([widgets_dict['test_distance'], widgets_dict['base_speed']]),
        widgets.HBox([widgets_dict['speed_scaling'], widgets_dict['convergence_threshold']]),
        widgets.HTML("<h3>🚀 Simulation Settings</h3>"),
        widgets_dict['max_steps'],
        widgets.HTML("<h3>📍 Starting Position</h3>"),
        widgets.HBox([widgets_dict['start_x'], widgets_dict['start_y']]),
        widgets_dict['start_orientation'],
        widgets.HTML("<hr>"),
        output
    ])
    
    return parameter_box

print("🎛️ Creating Interactive Parameter Exploration Interface")
print("====================================================\n")

# Create and display the interactive interface
exploration_interface = create_parameter_exploration_interface()
display(exploration_interface)

print("\n📝 Instructions:")
print("• Adjust the sliders above to modify algorithm parameters")
print("• Each change will automatically trigger a new simulation")
print("• Watch how different parameters affect navigation behavior")
print("• Green marker = start position, Red marker = end position")
print("• Try extreme values to understand parameter sensitivity")

## Parameter Sensitivity Analysis

Let's conduct a systematic analysis of how each parameter affects algorithm performance. This helps understand which parameters are most critical for successful odor navigation.

In [ ]:
def parameter_sensitivity_analysis(base_config: Dict, parameter_ranges: Dict, num_samples: int = 5):
    """
    Conduct systematic parameter sensitivity analysis.
    
    Tests each parameter across its range while keeping others constant,
    measuring the impact on navigation performance metrics.
    
    Args:
        base_config: Baseline configuration
        parameter_ranges: Dictionary of parameter names and their test ranges
        num_samples: Number of sample points for each parameter
    
    Returns:
        Dictionary containing sensitivity analysis results
    """
    print("🔬 Conducting Parameter Sensitivity Analysis")
    print("============================================\n")
    
    sensitivity_results = {}
    
    for param_name, (min_val, max_val) in parameter_ranges.items():
        print(f"📊 Analyzing parameter: {param_name}")
        print(f"   Range: {min_val} - {max_val}")
        
        # Generate test values
        test_values = np.linspace(min_val, max_val, num_samples)
        
        # Store results for this parameter
        param_results = {
            'values': test_values,
            'convergence_rate': [],
            'final_odor': [],
            'steps_to_convergence': [],
            'path_efficiency': [],
            'path_length': []
        }
        
        for test_value in test_values:
            # Create modified configuration
            test_config = base_config.copy() if hasattr(base_config, 'copy') else dict(base_config)
            
            # Update the specific parameter
            if param_name == 'test_distance':
                test_config['algorithm']['test_distance'] = test_value
            elif param_name == 'base_speed':
                test_config['algorithm']['base_speed'] = test_value
            elif param_name == 'speed_scaling':
                test_config['algorithm']['speed_scaling'] = test_value
            elif param_name == 'convergence_threshold':
                test_config['algorithm']['convergence_threshold'] = test_value
            
            # Run multiple trials for statistical robustness
            trial_results = []
            num_trials = 3  # Multiple trials with different seeds
            
            for trial in range(num_trials):
                # Set unique seed for each trial
                trial_seed = DEMO_SEED + trial * 1000 + hash(param_name) % 1000
                set_global_seed(trial_seed)
                
                try:
                    results = run_odor_following_demo(test_config, verbose=False)
                    trial_results.append(results)
                except Exception as e:
                    print(f"   ⚠️ Trial failed for {param_name}={test_value}: {e}")
                    continue
            
            if trial_results:
                # Calculate average metrics across trials
                convergence_rate = np.mean([r['converged'] for r in trial_results])
                final_odor = np.mean([r['final_odor'] for r in trial_results])
                steps = np.mean([r['total_steps'] for r in trial_results])
                
                # Calculate path metrics
                efficiencies = []
                lengths = []
                
                for r in trial_results:
                    trajectory = r['trajectory']
                    if len(trajectory) > 1:
                        path_length = np.sum(np.linalg.norm(np.diff(trajectory, axis=0), axis=1))
                        direct_distance = np.linalg.norm(r['final_position'] - trajectory[0])
                        efficiency = direct_distance / path_length if path_length > 0 else 0
                        efficiencies.append(efficiency)
                        lengths.append(path_length)
                
                avg_efficiency = np.mean(efficiencies) if efficiencies else 0
                avg_length = np.mean(lengths) if lengths else 0
                
                # Store metrics
                param_results['convergence_rate'].append(convergence_rate)
                param_results['final_odor'].append(final_odor)
                param_results['steps_to_convergence'].append(steps)
                param_results['path_efficiency'].append(avg_efficiency)
                param_results['path_length'].append(avg_length)
            else:
                # Handle failed trials
                param_results['convergence_rate'].append(0)
                param_results['final_odor'].append(0)
                param_results['steps_to_convergence'].append(test_config['algorithm']['max_steps'])
                param_results['path_efficiency'].append(0)
                param_results['path_length'].append(0)
        
        sensitivity_results[param_name] = param_results
        print(f"   ✅ Completed analysis for {param_name}")
    
    # Reset to original seed
    set_global_seed(DEMO_SEED)
    
    return sensitivity_results

# Define parameter ranges for sensitivity analysis
parameter_ranges = {
    'test_distance': (0.5, 4.0),
    'base_speed': (0.1, 0.8),
    'speed_scaling': (0.5, 4.0),
    'convergence_threshold': (0.6, 0.95)
}

# Run sensitivity analysis
sensitivity_data = parameter_sensitivity_analysis(
    demo_config, parameter_ranges, num_samples=7
)

In [ ]:
def visualize_sensitivity_analysis(sensitivity_data: Dict):
    """
    Create comprehensive visualization of parameter sensitivity analysis results.
    """
    n_params = len(sensitivity_data)
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    metrics = ['convergence_rate', 'final_odor', 'path_efficiency', 'steps_to_convergence']
    metric_labels = ['Convergence Rate', 'Final Odor Concentration', 'Path Efficiency', 'Steps to Convergence']
    metric_colors = ['green', 'blue', 'purple', 'orange']
    
    for i, (metric, label, color) in enumerate(zip(metrics, metric_labels, metric_colors)):
        ax = axes[i]
        
        for param_name, param_data in sensitivity_data.items():
            values = param_data['values']
            metric_values = param_data[metric]
            
            ax.plot(values, metric_values, 'o-', label=param_name, linewidth=2, markersize=6)
        
        ax.set_xlabel('Parameter Value (Normalized)')
        ax.set_ylabel(label)
        ax.set_title(f'Sensitivity Analysis: {label}', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend()
        
        # Add horizontal reference lines for some metrics
        if metric == 'convergence_rate':
            ax.axhline(0.5, color='red', linestyle='--', alpha=0.7, label='50% Success')
        elif metric == 'path_efficiency':
            ax.axhline(0.5, color='red', linestyle='--', alpha=0.7, label='50% Efficiency')
    
    plt.tight_layout()
    plt.show()
    
    # Print sensitivity summary
    print("\n📊 Parameter Sensitivity Summary:")
    print("=================================\n")
    
    for param_name, param_data in sensitivity_data.items():
        print(f"🔧 {param_name.upper()}:")
        
        # Calculate sensitivity metrics
        convergence_range = max(param_data['convergence_rate']) - min(param_data['convergence_rate'])
        efficiency_range = max(param_data['path_efficiency']) - min(param_data['path_efficiency'])
        
        print(f"  📈 Convergence rate range: {convergence_range:.2f}")
        print(f"  ⚡ Path efficiency range: {efficiency_range:.2f}")
        
        # Find optimal value (highest convergence rate)
        best_idx = np.argmax(param_data['convergence_rate'])
        optimal_value = param_data['values'][best_idx]
        optimal_convergence = param_data['convergence_rate'][best_idx]
        
        print(f"  🎯 Optimal value: {optimal_value:.2f} (convergence: {optimal_convergence:.2f})")
        print()
    
    # Overall recommendations
    print("💡 RECOMMENDATIONS:")
    print("===================\n")
    
    # Find most sensitive parameters
    sensitivity_scores = {}
    for param_name, param_data in sensitivity_data.items():
        convergence_sensitivity = max(param_data['convergence_rate']) - min(param_data['convergence_rate'])
        efficiency_sensitivity = max(param_data['path_efficiency']) - min(param_data['path_efficiency'])
        sensitivity_scores[param_name] = convergence_sensitivity + efficiency_sensitivity
    
    most_sensitive = max(sensitivity_scores, key=sensitivity_scores.get)
    least_sensitive = min(sensitivity_scores, key=sensitivity_scores.get)
    
    print(f"🎛️ Most sensitive parameter: {most_sensitive} (requires careful tuning)")
    print(f"🔧 Least sensitive parameter: {least_sensitive} (more robust to changes)")
    print(f"🎯 Focus tuning efforts on: {most_sensitive} and speed_scaling")
    print(f"⚖️ Balance between convergence rate and path efficiency for optimal performance")

# Visualize sensitivity analysis results
print("\n📈 Visualizing Parameter Sensitivity Analysis")
print("============================================\n")

visualize_sensitivity_analysis(sensitivity_data)

## Comparative Algorithm Analysis

Let's compare different variations of the gradient-following algorithm to understand how design choices affect performance.

In [ ]:
def compare_algorithm_variants():
    """
    Compare different variants of the odor-following algorithm.
    
    Tests various algorithmic approaches to understand the impact of
    design decisions on navigation performance.
    """
    print("🔄 Comparing Algorithm Variants")
    print("===============================\n")
    
    # Define algorithm variants
    variants = {
        'Standard (8-direction)': {
            'test_directions': [0, 45, 90, 135, 180, 225, 270, 315],
            'description': 'Tests 8 cardinal and diagonal directions'
        },
        'Cardinal Only (4-direction)': {
            'test_directions': [0, 90, 180, 270],
            'description': 'Tests only 4 cardinal directions (N, S, E, W)'
        },
        'High Resolution (16-direction)': {
            'test_directions': [i * 22.5 for i in range(16)],
            'description': 'Tests 16 directions for finer gradient detection'
        },
        'Forward-Biased': {
            'test_directions': [315, 0, 45, 90, 270],  # Emphasize forward directions
            'description': 'Biased toward forward movement directions'
        }
    }
    
    # Run comparison experiments
    comparison_results = {}
    
    for variant_name, variant_config in variants.items():
        print(f"🧪 Testing: {variant_name}")
        print(f"   {variant_config['description']}")
        
        # Create modified configuration
        test_config = demo_config.copy() if hasattr(demo_config, 'copy') else dict(demo_config)
        test_config['algorithm']['test_directions'] = variant_config['test_directions']
        
        # Run multiple trials
        trial_results = []
        num_trials = 5
        
        for trial in range(num_trials):
            trial_seed = DEMO_SEED + trial * 100
            set_global_seed(trial_seed)
            
            try:
                results = run_odor_following_demo(test_config, verbose=False)
                trial_results.append(results)
            except Exception as e:
                print(f"   ⚠️ Trial {trial} failed: {e}")
        
        if trial_results:
            # Calculate aggregate metrics
            convergence_rate = np.mean([r['converged'] for r in trial_results])
            avg_steps = np.mean([r['total_steps'] for r in trial_results])
            avg_final_odor = np.mean([r['final_odor'] for r in trial_results])
            
            # Path analysis
            path_lengths = []
            path_efficiencies = []
            
            for r in trial_results:
                trajectory = r['trajectory']
                if len(trajectory) > 1:
                    path_length = np.sum(np.linalg.norm(np.diff(trajectory, axis=0), axis=1))
                    direct_distance = np.linalg.norm(r['final_position'] - trajectory[0])
                    efficiency = direct_distance / path_length if path_length > 0 else 0
                    path_lengths.append(path_length)
                    path_efficiencies.append(efficiency)
            
            comparison_results[variant_name] = {
                'convergence_rate': convergence_rate,
                'avg_steps': avg_steps,
                'avg_final_odor': avg_final_odor,
                'avg_path_length': np.mean(path_lengths) if path_lengths else 0,
                'avg_path_efficiency': np.mean(path_efficiencies) if path_efficiencies else 0,
                'num_directions': len(variant_config['test_directions']),
                'description': variant_config['description'],
                'trial_results': trial_results
            }
            
            print(f"   ✅ Success rate: {convergence_rate:.2f}")
            print(f"   📊 Avg steps: {avg_steps:.1f}")
            print(f"   ⚡ Avg efficiency: {np.mean(path_efficiencies) if path_efficiencies else 0:.2f}")
        else:
            print(f"   ❌ All trials failed for {variant_name}")
        
        print()
    
    # Reset seed
    set_global_seed(DEMO_SEED)
    
    return comparison_results

def visualize_algorithm_comparison(comparison_results: Dict):
    """
    Create comprehensive visualization comparing algorithm variants.
    """
    # Prepare data for plotting
    variant_names = list(comparison_results.keys())
    metrics = {
        'Convergence Rate': [comparison_results[name]['convergence_rate'] for name in variant_names],
        'Average Steps': [comparison_results[name]['avg_steps'] for name in variant_names],
        'Path Efficiency': [comparison_results[name]['avg_path_efficiency'] for name in variant_names],
        'Final Odor': [comparison_results[name]['avg_final_odor'] for name in variant_names]
    }
    
    # Create comparison plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold']
    
    for i, (metric_name, values) in enumerate(metrics.items()):
        ax = axes[i]
        bars = ax.bar(variant_names, values, color=colors[i], alpha=0.7, edgecolor='black')
        
        # Add value labels on bars
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{value:.2f}', ha='center', va='bottom', fontweight='bold')
        
        ax.set_title(f'{metric_name} Comparison', fontsize=12, fontweight='bold')
        ax.set_ylabel(metric_name)
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add reference lines for some metrics
        if metric_name == 'Convergence Rate':
            ax.axhline(0.5, color='red', linestyle='--', alpha=0.7, label='50% Threshold')
            ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed comparison
    print("\n📊 Algorithm Variant Comparison Results:")
    print("========================================\n")
    
    # Sort variants by convergence rate
    sorted_variants = sorted(
        comparison_results.items(), 
        key=lambda x: x[1]['convergence_rate'], 
        reverse=True
    )
    
    for rank, (variant_name, results) in enumerate(sorted_variants, 1):
        print(f"🏆 Rank {rank}: {variant_name}")
        print(f"   📋 {results['description']}")
        print(f"   🎯 Convergence Rate: {results['convergence_rate']:.2f}")
        print(f"   📊 Average Steps: {results['avg_steps']:.1f}")
        print(f"   ⚡ Path Efficiency: {results['avg_path_efficiency']:.2f}")
        print(f"   👃 Final Odor: {results['avg_final_odor']:.3f}")
        print(f"   🧭 Directions Tested: {results['num_directions']}")
        print()
    
    # Analysis insights
    best_variant = sorted_variants[0]
    most_efficient = max(comparison_results.items(), key=lambda x: x[1]['avg_path_efficiency'])
    fastest = min(comparison_results.items(), key=lambda x: x[1]['avg_steps'])
    
    print("💡 ANALYSIS INSIGHTS:")
    print("====================\n")
    print(f"🏅 Best Overall: {best_variant[0]} (highest convergence rate)")
    print(f"⚡ Most Efficient: {most_efficient[0]} (best path efficiency)")
    print(f"🚀 Fastest: {fastest[0]} (fewest steps)")
    print()
    print("🔍 Key Observations:")
    print(f"• More directions generally improve gradient detection accuracy")
    print(f"• Trade-off between computational cost and navigation precision")
    print(f"• Forward-biased strategies may work well in certain environments")
    print(f"• 8-direction approach provides good balance of performance and efficiency")

# Run algorithm comparison
algorithm_comparison_results = compare_algorithm_variants()

# Visualize comparison results
if algorithm_comparison_results:
    print("\n📈 Visualizing Algorithm Comparison")
    print("===================================\n")
    visualize_algorithm_comparison(algorithm_comparison_results)
else:
    print("❌ No comparison results to visualize")

## Real-time Animation Demo

Finally, let's create an animated visualization showing the agent's navigation process in real-time, highlighting the decision-making process at each step.

In [ ]:
def create_animated_demo(config: Dict, save_animation: bool = False):
    """
    Create real-time animated demonstration of odor-following navigation.
    
    Shows the agent's movement and decision-making process with visual
    indicators for direction testing and gradient detection.
    """
    print("🎬 Creating Animated Navigation Demo")
    print("===================================\n")
    
    # Initialize navigator and environment
    navigator = SimpleOdorFollowingNavigator(config)
    environment = create_gaussian_odor_field(
        width=config['environment']['width'],
        height=config['environment']['height'],
        odor_sources=config['environment']['odor_sources']
    )
    
    # Set up visualization
    viz_config = {
        'animation': {
            'figsize': config['visualization']['figsize'],
            'interval': config['visualization']['animation_interval'],
            'fps': 5,
            'dpi': 100
        },
        'agents': {
            'trail_length': config['visualization']['trail_length'],
            'marker_size': 80,
            'trail_alpha': 0.7
        }
    }
    
    # Create visualization object
    try:
        if HYDRA_AVAILABLE:
            from omegaconf import OmegaConf
            viz_cfg = OmegaConf.create(viz_config)
        else:
            viz_cfg = viz_config
        
        simulation_viz = SimulationVisualization(
            config=viz_cfg,
            figsize=tuple(config['visualization']['figsize']),
            headless=False
        )
        
        # Setup environment
        simulation_viz.setup_environment(environment)
        
        print("✅ Visualization setup complete")
        
    except Exception as e:
        print(f"⚠️ Could not create SimulationVisualization: {e}")
        print("📊 Creating fallback matplotlib animation...")
        
        # Fallback to manual matplotlib animation
        fig, ax = plt.subplots(figsize=tuple(config['visualization']['figsize']))
        
        # Setup environment display
        im = ax.imshow(
            environment, origin='lower', cmap='viridis', alpha=0.8,
            extent=[0, environment.shape[1], 0, environment.shape[0]]
        )
        
        ax.set_title('Odor Following Navigation Demo', fontsize=14, fontweight='bold')
        ax.set_xlabel('X Position')
        ax.set_ylabel('Y Position')
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label('Odor Concentration')
        
        # Animation variables
        max_steps = config['algorithm']['max_steps']
        dt = config['algorithm']['time_step']
        trail_positions = []
        step_count = 0
        animation_data = []
        
        print(f"🚀 Starting navigation simulation...")
        print(f"📍 Initial position: {navigator.position}")
        
        # Run simulation and collect data
        for step in range(max_steps):
            step_info = navigator.step(environment, dt)
            decision = step_info['decision']
            
            # Store animation frame data
            frame_data = {
                'position': navigator.position.copy(),
                'orientation': navigator.orientation,
                'odor': decision['current_odor'],
                'step': step,
                'decision': decision,
                'converged': decision['converged']
            }
            animation_data.append(frame_data)
            
            print(f"📊 Step {step + 1}: Pos={navigator.position}, Odor={decision['current_odor']:.3f}")
            
            if decision['converged']:
                print(f"🎯 Convergence achieved at step {step + 1}!")
                break
        
        # Create static visualization showing the complete trajectory
        trajectory = np.array([frame['position'] for frame in animation_data])
        
        # Plot complete trajectory
        ax.plot(
            trajectory[:, 0], trajectory[:, 1], 
            'white', linewidth=3, alpha=0.9, label='Trajectory'
        )
        
        # Mark key positions
        ax.scatter(
            trajectory[0, 0], trajectory[0, 1], 
            color='green', s=150, marker='o', 
            edgecolors='black', linewidth=2, label='Start', zorder=10
        )
        
        ax.scatter(
            trajectory[-1, 0], trajectory[-1, 1], 
            color='red', s=150, marker='s', 
            edgecolors='black', linewidth=2, label='End', zorder=10
        )
        
        # Mark odor sources
        for i, (x, y, intensity, sigma) in enumerate(config['environment']['odor_sources']):
            ax.scatter(
                x, y, color='yellow', s=200, marker='*', 
                edgecolors='black', linewidth=2, 
                label=f'Odor Source {i+1}' if i == 0 else '', zorder=15
            )
        
        # Add trajectory annotations for key decision points
        annotation_steps = [0, len(animation_data)//4, len(animation_data)//2, len(animation_data)-1]
        
        for i, step_idx in enumerate(annotation_steps):
            if step_idx < len(animation_data):
                frame = animation_data[step_idx]
                pos = frame['position']
                odor = frame['odor']
                
                ax.annotate(
                    f'Step {step_idx + 1}\nOdor: {odor:.2f}',
                    xy=pos, xytext=(pos[0] + 2, pos[1] + 2),
                    fontsize=9, bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8),
                    arrowprops=dict(arrowstyle='->', color='black', alpha=0.7),
                    zorder=12
                )
        
        ax.legend(loc='upper left')
        
        plt.tight_layout()
        plt.show()
        
        # Print final statistics
        final_frame = animation_data[-1]
        print(f"\n📊 Navigation Complete!")
        print(f"📍 Final position: {final_frame['position']}")
        print(f"👃 Final odor concentration: {final_frame['odor']:.3f}")
        print(f"📊 Total steps: {len(animation_data)}")
        print(f"🎯 Converged: {'✅ YES' if final_frame['converged'] else '❌ NO'}")
        
        # Calculate path metrics
        path_length = np.sum(np.linalg.norm(np.diff(trajectory, axis=0), axis=1))
        direct_distance = np.linalg.norm(trajectory[-1] - trajectory[0])
        efficiency = direct_distance / path_length if path_length > 0 else 0
        
        print(f"📏 Path length: {path_length:.1f} units")
        print(f"📐 Direct distance: {direct_distance:.1f} units")
        print(f"⚡ Path efficiency: {efficiency:.2f}")
        
        return animation_data

# Create animated demonstration
animation_data = create_animated_demo(demo_config, save_animation=False)

print("\n🎉 Animation demo completed successfully!")
print("📝 You can adjust the demo_config parameters above and re-run this cell to see different behaviors.")

## Key Insights and Learning Summary

This notebook has demonstrated the core principles of gradient-following algorithms for odor plume navigation. Let's summarize the key insights and learning outcomes.

In [ ]:
# Generate comprehensive summary of findings
def generate_learning_summary():
    """
    Create a comprehensive summary of learning outcomes and insights.
    """
    print("📚 LEARNING SUMMARY: Odor Following Navigation Algorithms")
    print("=" * 60)
    print()
    
    print("🎯 KEY ALGORITHM PRINCIPLES:")
    print("============================\n")
    print("1. 🧭 GRADIENT DETECTION:")
    print("   • Test multiple directions simultaneously")
    print("   • Compare odor concentrations at test positions")
    print("   • Choose direction with highest concentration")
    print("   • 8-direction testing provides good balance of accuracy and efficiency")
    print()
    
    print("2. 🏃 ADAPTIVE SPEED CONTROL:")
    print("   • Base speed ensures continuous movement")
    print("   • Speed scaling amplifies gradient-based acceleration")
    print("   • Strong gradients → faster movement")
    print("   • Weak gradients → slower, more careful exploration")
    print()
    
    print("3. 🎯 CONVERGENCE CRITERIA:")
    print("   • Threshold-based stopping (e.g., odor > 0.9)")
    print("   • Prevents endless searching in strong odor regions")
    print("   • Balances exploration vs exploitation")
    print()
    
    print("4. 📏 BILINEAR INTERPOLATION:")
    print("   • Enables sub-pixel accuracy for odor sampling")
    print("   • Smoother navigation in discrete grid environments")
    print("   • Reduces discretization artifacts")
    print()
    
    print("⚡ PERFORMANCE INSIGHTS:")
    print("========================\n")
    print("🔧 CRITICAL PARAMETERS:")
    print("   • Test Distance: Affects gradient detection accuracy")
    print("   • Speed Scaling: Controls responsiveness to gradients")
    print("   • Convergence Threshold: Determines stopping behavior")
    print("   • Number of Test Directions: Balances accuracy vs computation")
    print()
    
    print("📊 ALGORITHM VARIANTS:")
    print("   • 4-direction: Fast but less accurate")
    print("   • 8-direction: Good balance (recommended)")
    print("   • 16-direction: Higher accuracy, more computation")
    print("   • Forward-biased: Useful for directed navigation")
    print()
    
    print("🌍 REAL-WORLD APPLICATIONS:")
    print("============================\n")
    print("🐛 BIOLOGICAL SYSTEMS:")
    print("   • Insect pheromone following")
    print("   • Marine animal chemical trail navigation")
    print("   • Bacterial chemotaxis")
    print()
    
    print("🤖 ROBOTICS APPLICATIONS:")
    print("   • Search and rescue operations")
    print("   • Environmental monitoring")
    print("   • Gas leak detection")
    print("   • Pollution source localization")
    print()
    
    print("🧠 MACHINE LEARNING:")
    print("   • Reinforcement learning environments")
    print("   • Multi-agent coordination")
    print("   • Exploration-exploitation strategies")
    print("   • Gradient-based optimization analogs")
    print()
    
    print("🔬 RESEARCH EXTENSIONS:")
    print("=======================\n")
    print("1. 📡 ADVANCED SENSING:")
    print("   • Multiple sensor arrays")
    print("   • Temporal gradient estimation")
    print("   • Noise-robust sensing strategies")
    print()
    
    print("2. 🤝 MULTI-AGENT COORDINATION:")
    print("   • Swarm-based exploration")
    print("   • Information sharing between agents")
    print("   • Distributed source localization")
    print()
    
    print("3. 🌪️ COMPLEX ENVIRONMENTS:")
    print("   • Turbulent flow effects")
    print("   • Multiple competing sources")
    print("   • Time-varying odor fields")
    print("   • Obstacles and boundaries")
    print()
    
    print("4. 🧠 LEARNING ALGORITHMS:")
    print("   • Adaptive parameter tuning")
    print("   • Experience-based strategy improvement")
    print("   • Transfer learning across environments")
    print()
    
    print("💡 PRACTICAL RECOMMENDATIONS:")
    print("==============================\n")
    print("🎛️ FOR PARAMETER TUNING:")
    print("   • Start with 8-direction testing")
    print("   • Test distance = 1-3x agent size")
    print("   • Speed scaling = 1.5-3.0 for responsive navigation")
    print("   • Convergence threshold = 0.8-0.95 depending on noise")
    print()
    
    print("🔧 FOR IMPLEMENTATION:")
    print("   • Use bilinear interpolation for smooth sampling")
    print("   • Implement boundary checks for test positions")
    print("   • Add noise tolerance for real-world robustness")
    print("   • Consider computational vs accuracy trade-offs")
    print()
    
    print("🎯 FOR RESEARCH:")
    print("   • Benchmark against multiple environment types")
    print("   • Compare with other navigation strategies")
    print("   • Analyze failure modes and edge cases")
    print("   • Validate with biological or physical data")
    print()
    
    print("🌟 CONCLUSION:")
    print("==============\n")
    print("Gradient-following algorithms provide a robust, intuitive approach to")
    print("odor source localization that scales from simple single-agent systems")
    print("to complex multi-agent swarms. The key to success lies in balancing")
    print("exploration breadth, computational efficiency, and environmental")
    print("adaptability through careful parameter tuning and algorithm design.")
    print()
    print("✨ The principles demonstrated here form the foundation for more")
    print("advanced navigation strategies and provide valuable insights into")
    print("both biological and artificial intelligence systems.")
    print()
    print("🚀 Ready to explore more advanced navigation algorithms!")

# Generate the comprehensive learning summary
generate_learning_summary()

print("\n" + "=" * 60)
print("🎓 Educational Demo Complete!")
print("📖 You've successfully explored gradient-following navigation algorithms")
print("🔬 Try modifying parameters above to deepen your understanding")
print("🤖 Ready to implement these concepts in your own research!")
print("=" * 60)